In [3]:
# For calculating the mutual inductance between 2 (soon 4) coils of wire
# Currently assumes N=1

import numpy as np
from scipy.integrate import cumtrapz, dblquad # scipy version = 1.7.3
import scipy

# Constants
mu_0 = 1.2566 * 10 ** (-6)        # Vacuum permeability in H/m

# coil constants (all in meters)
R_1 = 0.05
R_2 = 0.025
r = 0.03
z = 0.05

# calculate M

def f(phi2, phi1): # this is the Neumann formula for two circles with radii R_1 and R_2 with parallel axes, offset by r and z
    return np.cos(phi1 - phi2) / np.sqrt( R_1**2 + R_2**2 + r**2 + z**2 - 2*R_1*R_2*np.cos(phi1 - phi2) - 2*r*(R_1*np.sin(phi1) - R_2*np.sin(phi2)) )

M, M_error = mu_0 / (4 * np.pi) * np.array(dblquad(f, 0, 2*np.pi, lambda x: 0, lambda x: 2*np.pi))

print(f"M = {M}, error = {M_error}")

M = 5.011637732794961e-06, error = 1.0947014552717775e-14
